In [1]:
import os
import pandas as pd
from datetime import datetime
base_dir = "C://DATA//train//49_kaggle//KaggleFun//Sales//DATA"
cat = pd.read_csv(os.path.join(base_dir, "item_categories.csv"))
item = pd.read_csv(os.path.join(base_dir,'items.csv'))
sales = pd.read_csv(os.path.join(base_dir, 'sales_train.csv'))
sales = sales[sales["date_block_num"] < 33]
shops = pd.read_csv(os.path.join(base_dir, 'shops.csv'))
cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [2]:
monthly_sales = sales.groupby(["date_block_num","shop_id","item_id"])["item_cnt_day"].sum()
monthly_sales.head()

date_block_num  shop_id  item_id
0               0        32         6.0
                         33         3.0
                         35         1.0
                         43         1.0
                         51         2.0
Name: item_cnt_day, dtype: float64

In [3]:
ds = sales.pivot_table(index = ['shop_id','item_id', "item_price"],values = ['item_cnt_day'], columns = ['date_block_num'],fill_value = 0,aggfunc='sum')
ds.reset_index(inplace = True)
ds.head()

shop_id item_id item_price item_cnt_day                     \
date_block_num                                       0   1  2  3  4  5  6   
0                    0      30      265.0            0  31  0  0  0  0  0   
1                    0      31      434.0            0  11  0  0  0  0  0   
2                    0      32      221.0            6  10  0  0  0  0  0   
3                    0      33      347.0            3   3  0  0  0  0  0   
4                    0      35      247.0            1  14  0  0  0  0  0   

                ...                                
date_block_num  ... 23 24 25 26 27 28 29 30 31 32  
0               ...  0  0  0  0  0  0  0  0  0  0  
1               ...  0  0  0  0  0  0  0  0  0  0  
2               ...  0  0  0  0  0  0  0  0  0  0  
3               ...  0  0  0  0  0  0  0  0  0  0  
4               ...  0  0  0  0  0  0  0  0  0  0  

[5 rows x 36 columns]

In [4]:
ds.isnull().sum()

              date_block_num
shop_id                         0
item_id                         0
item_price                      0
item_cnt_day  0                 0
              1                 0
              2                 0
              3                 0
              4                 0
              5                 0
              6                 0
              7                 0
              8                 0
              9                 0
              10                0
              11                0
              12                0
              13                0
              14                0
              15                0
              16                0
              17                0
              18                0
              19                0
              20                0
              21                0
              22                0
              23                0
              24                0
              25   

In [5]:
item = item.set_index("item_id")
item.head()

,item_name,item_category_id
item_id,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40
1,!ABBYY FineReader 12 Professional Edition Full...,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40
4,***КОРОБКА (СТЕКЛО) D,40


In [6]:
item.loc[0]["item_category_id"]

40

In [7]:
ds["cat"] = ds.apply(lambda row: item.loc[row["item_id"].values[0]]["item_category_id"], axis=1)
ds.head()

shop_id item_id item_price item_cnt_day                     \
date_block_num                                       0   1  2  3  4  5  6   
0                    0      30      265.0            0  31  0  0  0  0  0   
1                    0      31      434.0            0  11  0  0  0  0  0   
2                    0      32      221.0            6  10  0  0  0  0  0   
3                    0      33      347.0            3   3  0  0  0  0  0   
4                    0      35      247.0            1  14  0  0  0  0  0   

                ...                            cat  
date_block_num  ... 24 25 26 27 28 29 30 31 32      
0               ...  0  0  0  0  0  0  0  0  0  40  
1               ...  0  0  0  0  0  0  0  0  0  37  
2               ...  0  0  0  0  0  0  0  0  0  40  
3               ...  0  0  0  0  0  0  0  0  0  37  
4               ...  0  0  0  0  0  0  0  0  0  40  

[5 rows x 37 columns]

In [8]:
cols = ds.columns.tolist()
cols = cols[-1:] + cols[:-1]
ds = ds[cols]
ds.head()

cat shop_id item_id item_price item_cnt_day                  \
date_block_num                                           0   1  2  3  4  5   
0               40       0      30      265.0            0  31  0  0  0  0   
1               37       0      31      434.0            0  11  0  0  0  0   
2               40       0      32      221.0            6  10  0  0  0  0   
3               37       0      33      347.0            3   3  0  0  0  0   
4               40       0      35      247.0            1  14  0  0  0  0   

                ...                                
date_block_num  ... 23 24 25 26 27 28 29 30 31 32  
0               ...  0  0  0  0  0  0  0  0  0  0  
1               ...  0  0  0  0  0  0  0  0  0  0  
2               ...  0  0  0  0  0  0  0  0  0  0  
3               ...  0  0  0  0  0  0  0  0  0  0  
4               ...  0  0  0  0  0  0  0  0  0  0  

[5 rows x 37 columns]

In [ ]:
for i, row in enumerate(ds.iterrows()):
    if i == 0:
        print(row[1]["item_cnt_day"][0:4].tolist())
        print(row[1]["shop_id"].values[0])

In [27]:
from tqdm import tqdm
import numpy as np
def ds2ds(ds, lag=10):
    new_ds = []
    for row in tqdm(ds.iterrows()):
        for i in range(33-lag):
            time = row[1]["item_cnt_day"][i:i+lag].tolist()
            temp_list = [row[1]["shop_id"].values[0], row[1]["item_id"].values[0], row[1]["item_price"].values[0], row[1]["cat"].values[0]]
            temp_list += time
            new_ds.append(temp_list)
    return np.array(new_ds)

In [28]:
new_ds = ds2ds(ds)
print(len(new_ds))

723869it [7:45:49, 25.90it/s]


16648987


In [29]:
import pickle
pickle.dump( new_ds, open( os.path.join(base_dir,"train_data.pkl"), "wb" ) )

In [30]:
sales_test = pd.read_csv(os.path.join(base_dir, 'sales_train.csv'))
sales_test = sales_test[sales_test["date_block_num"] > 23]

sales_test = sales_test.pivot_table(index = ['shop_id','item_id', "item_price"],values = ['item_cnt_day'], columns = ['date_block_num'],fill_value = 0,aggfunc='sum')
sales_test.reset_index(inplace = True)
sales_test.head()

shop_id item_id item_price item_cnt_day                       \
date_block_num                                      24 25 26 27 28 29 30 31   
0                    2      31      399.0            0  0  0  0  0  0  0  0   
1                    2      32      149.0            0  0  1  0  0  0  0  1   
2                    2      33      199.0            0  1  0  1  1  0  1  0   
3                    2      53      299.0            0  1  0  0  0  0  0  0   
4                    2      59      249.0            0  1  0  0  0  0  0  0   

                      
date_block_num 32 33  
0               0  1  
1               0  0  
2               1  0  
3               0  0  
4               0  0

In [31]:
sales_test["cat"] = sales_test.apply(lambda row: item.loc[row["item_id"].values[0]]["item_category_id"], axis=1)

cols = sales_test.columns.tolist()
cols = cols[-1:] + cols[:-1]
sales_test = sales_test[cols]
sales_test.head()

cat shop_id item_id item_price item_cnt_day                    \
date_block_num                                          24 25 26 27 28 29 30   
0               37       2      31      399.0            0  0  0  0  0  0  0   
1               40       2      32      149.0            0  0  1  0  0  0  0   
2               37       2      33      199.0            0  1  0  1  1  0  1   
3               57       2      53      299.0            0  1  0  0  0  0  0   
4               57       2      59      249.0            0  1  0  0  0  0  0   

                         
date_block_num 31 32 33  
0               0  0  1  
1               1  0  0  
2               0  1  0  
3               0  0  0  
4               0  0  0

In [32]:
def ds2test(ds, lag = 10):
    new_ds = []
    for row in ds.iterrows():
        temp_list = [row[1]["shop_id"].values[0], row[1]["item_id"].values[0], row[1]["item_price"].values[0], row[1]["cat"].values[0]]
        temp_list += row[1]["item_cnt_day"][0:lag].tolist()
        new_ds.append(temp_list)
    return new_ds

ds_test = ds2test(sales_test)

In [33]:
ds_test.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
from torch.utils.data import Dataset, DataLoader
class NumpyDataset(Dataset):
    def __init__(self, ds):
        self.x_cat = ds[:,0:4]
        self.x_time = ds[:,5:-1]
        self.y = ds[:,-1]
    def __len__(self): 
        return len(self.y)
    def __getitem__(self, idx):
        return self.x_cat[idx], self.x_time[idx], self.y[idx]

In [ ]:
train_ds = NumpyDataset(new_ds)
test_ds = NumpyDataset(ds_test)

batch_size = 200

train_loader = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last = True)
test_loader = DataLoader(test_ds, shuffle=True, batch_size=batch_size, drop_last = True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.lstm_layers = nn.Sequential(
            nn.LSTM(10,16),
            nn.LSTM(16,32),
            nn.Dropout(0.5),
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4, 16),
            nn.Linear(16,32),
            nn.Dropout(0.25),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(64,32),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            nn.Linear(32,1),
        )

    # Defining the forward pass    
    def forward(self, x1, x2):
        x1 = self.linear_layers(x1)
        x2 = self.lstm_layers(x2)
        x = torch.cat([x1, x2], axis = 1)
        x = self.fc(x)
        return x

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Net()
model.to(device)
lr=0.05
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 100
print(model)

In [ ]:
model.train()
loss_min = np.inf
for i in range(epochs):
    for batch_idx, (x1, x2, y) in enumerate(train_loader):
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        model.zero_grad()
        output = model(x1, x2)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        if batch_idx == 0:
            test_acc = []
            for xx1, xx2, yy in test_loader:
                xx1, xx2, yy = xx1.to(device), xx2.to(device), yy.to(device)
                model.zero_grad()
                yyout = model(xx1, xx2)
                test_acc.append(criterion(yyout, yy).detach().cpu().numpy())
            print("Epoch: {}, Loss: {:.2f}, Val Acc: {:.2f}".format(i, loss, np.mean(test_acc)))

In [ ]:
# https://www.kaggle.com/karanjakhar/simple-and-easy-aprroach-using-lstm